In [6]:
import pandas as pd
import mysql.connector
import numpy as np

# Leer CSV limpio
df = pd.read_csv("../data/limpio/establecimientos_educativos_limpio.csv", encoding="utf-8-sig")

print("Columnas del CSV:", df.columns.tolist())

# Mapeo para convertir nombres de columnas limpios a tipos SQL
tipo_sql_por_col = {
    "anio": "VARCHAR(10)",
    "secretaria": "VARCHAR(255)",
    "codigodepartamento": "INT",
    "nombredepartamento": "VARCHAR(255)",
    "codigomunicipio": "INT",
    "nombremunicipio": "VARCHAR(255)",
    "codigoestablecimiento": "BIGINT",
    "nombreestablecimiento": "VARCHAR(255)",
    "zona": "VARCHAR(50)",
    "direccion": "VARCHAR(255)",
    "telefono": "VARCHAR(50)",
    "nombre_rector": "VARCHAR(255)",
    "tipo_establecimiento": "VARCHAR(100)",
    "etnias": "TEXT",
    "niveles": "VARCHAR(255)",
    "jornadas": "VARCHAR(255)",
    "especialidad": "VARCHAR(255)",
    "grados": "VARCHAR(255)",
    "modelos_educativos": "VARCHAR(500)",
    "capacidades_excepcionales": "VARCHAR(255)",
    "discapacidades": "VARCHAR(500)",
    "idiomas": "VARCHAR(255)",
    "numero_de_sedes": "INT",
    "prestador_de_servicio": "VARCHAR(255)",
    "propiedad_planta_fisica": "VARCHAR(100)",
    "resguardo": "VARCHAR(100)",
    "matricula_contratada": "VARCHAR(100)",
    "calendario": "VARCHAR(50)",
    "internado": "VARCHAR(50)",
    "estrato_socio_economico": "VARCHAR(50)",
    "correo_electronico": "VARCHAR(255)"
}

# Ordenar columnas según las presentes en el CSV
columnas_ordenadas = [col for col in tipo_sql_por_col.keys() if col in df.columns]

# Crear sentencia CREATE TABLE incluyendo la columna id
column_defs = ["id INT AUTO_INCREMENT PRIMARY KEY"]
for col in columnas_ordenadas:
    tipo_sql = tipo_sql_por_col.get(col, "VARCHAR(255)")
    column_defs.append(f"{col} {tipo_sql}")

create_table_sql = f"""
CREATE TABLE IF NOT EXISTS establecimientos_educativos (
    {', '.join(column_defs)}
);
"""

print("SQL para crear tabla:")
print(create_table_sql)

# Conexión a MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="edu_col"
)
cursor = conn.cursor()

# Crear tabla
cursor.execute(create_table_sql)

# Reordenar DataFrame según columnas
df = df[columnas_ordenadas]

# Preparar query de inserción (excluye columna id)
insert_query = f"""
INSERT INTO establecimientos_educativos(
    {', '.join(columnas_ordenadas)}
) VALUES ({', '.join(['%s'] * len(columnas_ordenadas))})
"""

# Insertar datos
for row in df.itertuples(index=False):
    clean_row = tuple(None if (isinstance(x, float) and np.isnan(x)) else x for x in row)
    cursor.execute(insert_query, clean_row)

conn.commit()
cursor.close()
conn.close()

print("Datos importados correctamente.")

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
    --------------------------------------- 0.3/16.4 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.4 MB 2.2 MB/s eta 0:00:07
   -- ------------------------------------- 1.0/16.4 MB 2.2 MB/s eta 0:00:08
   --- ------------------------------------ 1.6/16.4 MB 2.0 MB/s eta 0:00:08
   ----- ---------------------------------- 2.4/16.4 MB 2.2 MB/s eta 0:00:07
   ------- -------------------------------- 2.9/16.4 MB 2.4 MB/s eta 0:00:06
   -------- ------------------------------- 3.7/16.4 MB 2.6 MB/s eta 0:00:05
   ----------- ---------------------------- 4.7/16.4 MB 2.8 MB/s eta 0:00:05
   ------------- -------------------------- 5.5/16.4 MB 2.9 MB/s eta 0:00:04
   -------------- ------------------------- 6.0/16.4 MB 3.0 MB/s eta 0:00:04
   ---------------- ----------------------- 6.8/16.4 MB 3.0 MB/s eta 0:00:04
   ---------